In this notebook I will try to create a medical specialty chooser in which the patient introduces his symptoms and the model will tell the medical specialty that the patient should go to.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
)
from langchain.callbacks import get_openai_callback
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [2]:
# Desired output format.
class MedicalSpecialty(BaseModel):
    medical_specialty: str = Field(description="medical specialty the patient should go to")
    urgent: bool = Field(description="the patient should go to the hospital immediately")
 
parser = PydanticOutputParser(pydantic_object=MedicalSpecialty)

# system message prompt. It defines the instructions for the system.
template="""You are a medical expert.
            You will receive the symptoms and will tell the medical specialty the patient should go to.
            {format_instructions}"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,human_message_prompt]
)


In [3]:
queries = ["i have ache in my chest and in my left arm.",
           "me duele mucho la cabeza en la parte frontal y detrás, sudo mucho y estoy mareado.",
           "dolor abdominal en mi parte derecha que se irradia a la espalda.fiebre.",
           "me duele un diente.",
           "cuando como pan voy mucho a hacer caca.",
           "no hago pipí desde hace 12 horas."]

with get_openai_callback() as cb:
    for query in queries:
        llm = ChatOpenAI(temperature=0)

        chain = LLMChain(llm=llm, prompt=chat_prompt)
        print(query)
        print(chain.run({'text': query, 'format_instructions': parser.get_format_instructions()}))
        print("====================================")
        
    # print te costs of the requests
    print(cb)



i have ache in my chest and in my left arm.
{"medical_specialty": "Cardiology", "urgent": true}
me duele mucho la cabeza en la parte frontal y detrás, sudo mucho y estoy mareado.
{"medical_specialty": "Neurology", "urgent": false}
dolor abdominal en mi parte derecha que se irradia a la espalda.fiebre.
{"medical_specialty": "Gastroenterology", "urgent": true}
me duele un diente.
{"medical_specialty": "Dentistry", "urgent": false}
cuando como pan voy mucho a hacer caca.
{"medical_specialty": "Gastroenterology", "urgent": false}
no hago pipí desde hace 12 horas.
{"medical_specialty": "Urology", "urgent": true}
Tokens Used: 1555
	Prompt Tokens: 1464
	Completion Tokens: 91
Successful Requests: 6
Total Cost (USD): $0.00311
